In [41]:
from itertools import product
from random import random, randint, shuffle, seed, choices
import numpy as np
from scipy import sparse
from copy import copy
from functools import reduce

In [42]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge
Find the best solution with the fewest calls to the fitness functions for:
* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [246]:
NUM_POINTS = NUM_SETS = 1000
x = make_set_covering_problem(NUM_POINTS, NUM_SETS, .7)
print("Element at row=42 and column=42:", x[42, 42])
set1 = x[[0], :].toarray()
set2 = x[[1], :].toarray()


Element at row=42 and column=42: True


In [223]:
def covered(state):
    return np.sum(
        reduce(
            np.logical_or,
            [x[[i],:].toarray() for i,taken in enumerate(state) if taken],
            np.array([False for _ in range(NUM_SETS)])))

def tweak(state):
    new_state = copy(state)
    index = randint(0, NUM_SETS - 1)
    new_state[index] = not new_state[index]
    return new_state

def fitness(state):
    cost = sum(state)
    num_values_covered = covered(state)
    return num_values_covered, -cost

In [247]:
starting_state = [random() < 0 for _ in range(NUM_SETS)]
best = fitness(starting_state)
print(fitness(starting_state))
state = starting_state

for i in range(2000):
    new_state = tweak(state)
    new = fitness(new_state)
    if new >= best:
        best = new
        state = new_state
        #print(fitness(state))
        count_fitness = i + 1

_,n_sets = fitness(state)
print(f"Solution found with {count_fitness} fitness call and using {-n_sets} SETS")


(0, 0)
Solution found with 639 fitness call and using 5 SETS


# Simulated Annealing

In [225]:
def f(n,sets):
    if sets == 0:
        return 0
    else:
        return n ** 3/2 - sets

In [248]:
starting_state = [random() < 0 for _ in range(NUM_SETS)]
c = 0 #counter

t_0 = 100
t = t_0
final_t = 1e-20
alpha = 0.95

print(fitness(starting_state))
state = starting_state
for k in range(20000):
    c += 1
    
    new_state = tweak(state)
    f_new = fitness(new_state)
    f_old = fitness(state)
    fn = f(f_new[0],-f_new[1])
    fo = f(f_old[0],-f_old[1])
    if fn >= fo:
        state = new_state
        #print(fitness(state))
        count_fitness = k+1
        c = 0
    elif t > 0:
        
        p = np.exp(-(fo-fn) / t)
        if random() < p:
            state = new_state
            count_fitness = k+1
            c = 0
            #print(f"{p} && {t} && {(fo-fn)}----{fitness(state)}")
            #print(f"-{fitness(state)}")
    if t > final_t:
        t *= alpha
        

n,n_sets = fitness(state)
print(f"Solution found with {count_fitness} fitness call and using {-n_sets} SETS")
print(n)


(0, 0)
Solution found with 3179 fitness call and using 5 SETS
1000


In [48]:
covered(state)

100